**Importando bibliotecas**

In [ ]:
import pandas as pd
import numpy as np

**Obtendo dados**

In [ ]:
#Python
try:
    print('Obtendo dados...')

ENDERECO_DADOS = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv'

df = pd.read_csv(ENDERECO_DADOS, sep= ',', encoding='utf-8')[['review_title', 'overall_rating']]
                                                            
df = df.dropna(subset=['review_title','overall_ratting'])    

# excluindo dados não existentes (NaN)\n",
df = df.dropna(subset=['review_title','overall_rating'])

#Tranformando colunas em arrays,
texts = np.array(df['review_title'])
rating = np.array(df['overall_rating'])

print(df.head())
    
    except Exception as e:
    print('Erro ao obter dados:',e)







**Vetorização**

In [ ]:
# Biblioteca para trabalhar com redes neurais artificiais
#Tensorflow - https://www.tensorflow.org/?hl=pt-br# Tokenizar
    from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
    # ajustar o tamanho do vetor
    from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore

try:
    print('Vetorizando texto...')
    
    # Passo 1: tokenizar
    tokenizer = Tokenizer()

    # Passo 2: Criar o dicionário\n",
    # fit_on_texts: Cria o vocabulário, através do dicionário\n",
    # associando cada token a um índice numérico\n",
    # lembrando que se a palavra aparecer mais de uma vez, ela vai receber o mesmo índice numérico\n",
    tokenizer.fit_on_texts(texts)

    # Passo 3: Vetorizar, ou seja, transformar os tokens em números,\n",
    # a partir do dicionário criado no passo 2
    vetores = tokenizer.texts_to_sequences(texts)
    
    # Passo4: Padronização do tamanho do vetor - pad\n",
    padded_vetores = pad_sequences(vetores)
    
    print(padded_vetores)
   
    print('Textos vetorizados!')
 
except Exception as e:
        print('Erro ao vetorizar textos: ', e)


**Rede Neural**

In [ ]:
# Definição do modelo de rede neural utilizada
from tensorflow.keras.models import Sequential # type: ignore
# Camadas da rede neural

from tensorflow.keras.layers import Embedding, LSTM, Dense # type: ignore


try:
    print('Construindo a rede neural')

# Constante do modelo
# 1ª constante: tamanho do vocabulário

VOCAB_SIZE = len(tokenizer.word_index) + 1     

# 2ª constante: tamanho máximo da sequência
# é o comprimento máximo de um texto

MAX_SEQUENCE_LENGHT = padded_vetores.shape[1]

# 3ª constante: tamanho do vetor de entrada
# A literatura recomenda que inicia-se por uma quantidade igual a raiz quadrada do tamanho do vocabulário
# se o volume de dados for de larga escala, pode testar iniciando com um tamanho maior
# se o volume de dados for muito pequeno, pode-se testar iniciando com um tamanho menor
# Cuidado com o overfitting, que é quando o modelo aprende demais e começa a perde a capacidade de generalizar melhor, ou seja, observar todas as diferenças textuais
# Overfitting pode ser observado no treino da rede neural

VETOR_LENGHT = int(np.sqrt(VOCAB_SIZE))

# inicia-se a construção da rede neural
# sequential é um fluxo linear de camadas( conforme visto na aula 02 RNA.pptx)
# são processadas em ordem

model = Sequential()

# camada de entrada
# Embedding, na qual os vetores de texto são inseridos

model.add(Embedding(input_dim= VOCAB_SIZE,output_dim=VETOR_LENGHT, input_lenght = MAX_SEQUENCE_LENGHT))
                
# Camada oculta ou intermediária
# LSTM - long short-term memory, em portugues é "memória de longo e curto prazo"
# é onde a magia acontece, ou seja, onde o modelo treina baseado nos seus vetores
# números de unidades de memória, que basicamente é a quantidade de neurônios
# primeiro TESTE, experimente somente com uma camada! cuidado com overfitting!
# se for necessário adcionar mais camadas basta adicionar repetir o comando abaixo

# primeira camada oculta

model.add(LSTM(128))

# se é necessário adicionar outra camada oculta
# model.add(LSTM(64))

# camada de saida que é a camada densa
# regressão que é o caso desse exemplo. Somente 1 camada

model.add(Dense(1))

# construir o modelo
# é literalmente pegar as definições anteriores e construir o modelo
# input_shape: é p formato do dados de entrada e ainda o tamanho maximo do meu texto(MAX_SEQUENCE_LENGHT)

model.build(input_shape= (None,MAX_SEQUENCE_LENGHT))

# otimizador de taxa de aprendizado
# importante para ajustar, em casos de overfitting
# Adam: é nosso otimizador que vai ajustar essa taxa de aprendizado
# learning_rate: quanto menor


# Otimizador de taxa de aprendizado\n",
# importante para ajustar, em casos de overfitting\n",
# Adam: É nosso otimizador que vaia ajustar essa taxa de aprendizado\n",
# learning_rate: Quanto menor, melhor o aprendizado. Menos risco de overfitting\n",
otimizador = Adam(learning_rate=0.0005)
   
# compilar o modelo\n",
# Verificar se possui algum erro ou se tá de boa\n",
# Além disso, vamos informar o otimizador e a nossa métrica de perda (loss)\n",
# loss - Erro quadrado médio (mean_squared_error)\n",
model.compile(optimizer=otimizador, loss='mean_squared_error')

model.summary()
print('Modelo configurado e criado')
   

except Exception as e:
    print('Erro ao construir a rede neural', e)

In [ ]:
# EXPLICAR MELHOR NA PRÓXIMA AULA\n",
from sklearn.model_selection import train_test_split
   
try:
        print('Treinar o modelo de rede neural')
   
X_train, X_test, y_train, y_test =train_test_split(padded_vetores,rating,test_size=0.2, random_state=42)
    
# o treino da rede neural\n",
model.fit (X_train,y_train,epochs=5,batch_size=128,validation_data=(X_test, y_test))


except Exception as e:
       print('Erro ao treinar a rede neural: ', e)

In [ ]:
novos_textos = ['Muito bom gostei bastante. Top demmais! compensa muito!','Não recomendo, péssimo produto. Não funciona']